In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'Bahia - Consumo de Cimento (t)'
start_index = 0
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [8]:
data = pd.read_csv('2003_mo_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Bahia - value,Bahia - IDH Educacao,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - IDH Renda,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - Consumo de Cimento (t)
0,2003-1,0.299858,0.598394,0.766362,0.669899,39.798880,0.638715,1.317344e+08,8.384593e+06,8.566149,...,1.921789e+10,8.202881e+10,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,151.297
1,2003-2,0.301903,0.598795,0.766745,0.670210,39.480034,0.638948,1.318964e+08,8.391946e+06,8.569210,...,1.921789e+10,8.222716e+10,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,138.707
2,2003-3,0.303709,0.599196,0.767127,0.670521,39.400256,0.639181,1.320584e+08,8.399299e+06,8.572270,...,1.921789e+10,8.234287e+10,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,135.009
3,2003-4,0.305311,0.599597,0.767509,0.670831,39.417185,0.639414,1.322204e+08,8.406652e+06,8.575331,...,1.921789e+10,8.235526e+10,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,126.554
4,2003-5,0.306860,0.599998,0.767892,0.671142,39.479943,0.639647,1.323824e+08,8.414005e+06,8.578392,...,1.921843e+10,8.245980e+10,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,137.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2022-9,0.596178,NaN,NaN,NaN,105.752943,0.659315,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,346.042
237,2022-10,0.594662,NaN,NaN,NaN,105.537361,0.659395,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,347.901
238,2022-11,0.592436,NaN,NaN,NaN,104.971241,0.659476,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310.845
239,2022-12,0.589305,NaN,NaN,NaN,104.857141,0.659557,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310.845


In [9]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Bahia - value,Bahia - IDH Educacao,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - IDH Renda,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-1.436732,-1.890989,-2.630882,-2.195846,-1.793459,-2.210000,-1.766933,-0.695996,-2.315693,-2.238131,...,1.155957,-1.063593,-0.822933,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-1.404185,-1.868832,-2.578239,-2.161313,-1.806616,-2.177528,-1.747868,-0.654222,-2.273375,-2.196868,...,1.155957,-1.063593,-0.818174,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-1.375442,-1.846676,-2.525597,-2.126781,-1.809908,-2.145055,-1.728803,-0.612447,-2.231056,-2.155605,...,1.155957,-1.063593,-0.815398,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-1.349941,-1.824519,-2.472955,-2.092248,-1.809210,-2.112583,-1.709739,-0.570673,-2.188738,-2.114342,...,1.155957,-1.063593,-0.815101,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-1.325290,-1.802362,-2.420313,-2.057715,-1.806620,-2.080111,-1.690674,-0.528898,-2.146420,-2.073079,...,1.151943,-1.063566,-0.812592,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.851312,1.208211,0.651753,1.041952,0.619029,0.966650,0.865987,-1.416165,0.555618,0.463875,...,3.115817,1.690001,0.667867,-0.689886,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.873607,1.201241,0.624227,1.026783,0.635352,0.964148,0.848481,-1.404857,0.531641,0.443334,...,3.295619,1.717926,0.640959,-0.441954,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.901718,1.194270,0.596702,1.011615,0.652040,0.961646,0.830975,-1.393550,0.507664,0.422793,...,3.486686,1.748758,0.615514,-0.132782,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.928136,1.187300,0.569176,0.996447,0.638794,0.959144,0.813469,-1.382243,0.483687,0.402252,...,3.692016,1.780444,0.593546,0.084061,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      138.707
1      135.009
2      126.554
3      137.331
4      118.680
        ...   
236    347.901
237    310.845
238    310.845
239        NaN
240        NaN
Name: Bahia - Consumo de Cimento (t), Length: 241, dtype: float64

In [11]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Bahia - value,Bahia - IDH Educacao,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - IDH Renda,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-1.436732,-1.890989,-2.630882,-2.195846,-1.793459,-2.210000,-1.766933,-0.695996,-2.315693,-2.238131,...,1.155957,-1.063593,-0.822933,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-1.404185,-1.868832,-2.578239,-2.161313,-1.806616,-2.177528,-1.747868,-0.654222,-2.273375,-2.196868,...,1.155957,-1.063593,-0.818174,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-1.375442,-1.846676,-2.525597,-2.126781,-1.809908,-2.145055,-1.728803,-0.612447,-2.231056,-2.155605,...,1.155957,-1.063593,-0.815398,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-1.349941,-1.824519,-2.472955,-2.092248,-1.809210,-2.112583,-1.709739,-0.570673,-2.188738,-2.114342,...,1.155957,-1.063593,-0.815101,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-1.325290,-1.802362,-2.420313,-2.057715,-1.806620,-2.080111,-1.690674,-0.528898,-2.146420,-2.073079,...,1.151943,-1.063566,-0.812592,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.005306,1.275616,1.370224,1.339087,0.469766,1.171052,1.163205,-1.340370,1.005749,1.008566,...,-0.660781,1.272681,1.521503,-0.742853,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806
164,1.029532,1.275493,1.358762,1.336451,0.457435,1.163001,1.164562,-1.364998,1.008863,1.002613,...,-0.660781,1.288866,1.512161,-0.787367,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523
165,1.057198,1.275369,1.347300,1.333816,0.454716,1.154950,1.165920,-1.389626,1.011977,0.996661,...,-0.660781,1.304998,1.500026,-0.757940,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175
166,1.082730,1.275246,1.335837,1.331180,0.446427,1.146900,1.167277,-1.414254,1.015092,0.990708,...,-0.660781,1.332157,1.486995,-0.917469,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146


In [12]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      138.707
1      135.009
2      126.554
3      137.331
4      118.680
        ...   
163    280.074
164    277.938
165    275.802
166    273.666
167    271.530
Name: Bahia - Consumo de Cimento (t), Length: 168, dtype: float64

In [13]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Bahia - value,Bahia - IDH Educacao,Bahia - IDH Longevidade,Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - IDH Renda,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
168,1.142199,1.274999,1.312912,1.325909,0.407445,1.130798,1.169992,-1.463511,1.021321,0.978804,...,-0.660781,1.364625,1.460212,-1.032458,-0.585256,-0.505289,-1.202612,1.035545,0.707473,-1.446117
169,1.169669,1.275283,1.296236,1.318823,0.400025,1.131519,1.166430,-1.470219,1.023238,0.968444,...,-0.660781,1.371257,1.445627,-1.077854,-0.581056,-0.373612,-1.139987,1.046449,0.692399,-1.526274
170,1.197459,1.275567,1.279560,1.311738,0.398398,1.132241,1.162868,-1.476927,1.025155,0.958085,...,-0.660781,1.375328,1.430008,-1.036724,-0.574769,-0.291560,-1.053534,1.057352,0.677324,-1.610341
171,1.231686,1.275851,1.262884,1.304652,0.379182,1.132962,1.159306,-1.483635,1.027072,0.947726,...,-0.660781,1.381257,1.413730,-1.013400,-0.567021,-0.450118,-0.932974,1.068256,0.662249,-1.601301
172,1.268038,1.276135,1.246208,1.297567,0.384352,1.133683,1.155744,-1.490343,1.028989,0.937367,...,-0.660781,1.364882,1.396980,-1.059228,-0.557842,-0.909054,-0.763756,1.079159,0.647175,-1.618284
173,1.299588,1.276419,1.229531,1.290482,0.367059,1.134404,1.152182,-1.497051,1.030906,0.927007,...,-0.582593,1.369812,1.379116,-1.170032,-0.548012,0.387442,-0.644652,1.090063,0.632100,-1.602348
174,1.338328,1.276703,1.212855,1.283396,0.363065,1.135125,1.148620,-1.503759,1.032823,0.916648,...,-0.474908,1.376543,1.360141,-1.115621,-0.537244,-0.937589,-0.541601,1.100966,0.617026,-1.686659
175,1.376421,1.276987,1.196179,1.276311,0.360415,1.135846,1.145058,-1.510467,1.034740,0.906289,...,-0.364364,1.384496,1.341361,-1.182159,-0.524149,-1.212466,-0.565747,1.111870,0.601951,-1.688184
176,1.409717,1.277271,1.179503,1.269226,0.359113,1.136567,1.141496,-1.517175,1.036658,0.895930,...,-0.250935,1.391358,1.323058,-1.270623,-0.519647,-0.499856,-0.536256,1.122773,0.586877,-1.692173
177,1.441251,1.277555,1.162827,1.262140,0.367682,1.137288,1.137934,-1.523883,1.038575,0.885571,...,-0.134596,1.399740,1.303196,-1.321931,-0.510389,-0.120028,-0.544036,1.133677,0.571802,-1.690760


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

168    213.266
169    285.938
170    219.576
171    267.203
172    240.714
173    250.101
174    277.528
175    270.092
176    278.146
177    257.458
178    231.748
179    268.336
180    223.453
181    241.464
182    238.901
183    191.989
184    272.452
185    261.009
186    292.688
187    258.881
188    276.879
189    255.774
190    208.326
191    291.428
192    249.430
193    241.612
194    252.303
195    281.912
196    200.213
197    270.511
198    281.466
199    285.535
200    328.259
201    298.078
202    264.838
203    300.311
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [15]:
def validation_splitter(df, div_factor, add_factor):
    split_factor = len(df)//div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop_index.append(pos + add_factor)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop_index), df.iloc[positions_to_drop]

In [16]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6, 0)
    target,target_val = validation_splitter(train_target, 6, 0)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2360793833, 2046932901, 1032176266, 2073564254, 1190890854, 3468556023, 2027863536, 3516736498, 457782744, 825139877, 1677325863, 1069152882, 336240673, 3668977071, 4254347119, 1972142621, 301492780, 917021409, 1574993953, 3468524231, 3282773125, 3794823208, 685376338, 601039690, 3553825260]


Step: 0 ___________________________________________
val_loss: 138.3181915283203
winner_seed: 2360793833


Step: 1 ___________________________________________
val_loss: 109.01384735107422
winner_seed: 2046932901


Step: 2 ___________________________________________
val_loss: 99.66685485839844
winner_seed: 1032176266


Step: 3 ___________________________________________
val_loss: 136.27853393554688


Step: 4 ___________________________________________
val_loss: 111.09034729003906


Step: 5 ___________________________________________
val_loss: 148.58641052246094


Step: 6 ___________________________________________
val_loss: 161.1203155517578


Step: 7 ___________________________________________
va

In [21]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 22ms/step - loss: 65943.7266 - val_loss: 36080.3359
Epoch 2/10000
5/5 [==============================] - 0s 4ms/step - loss: 43171.9297 - val_loss: 43277.8477
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 33185.9375 - val_loss: 21928.1738
Epoch 4/10000
5/5 [==============================] - 0s 4ms/step - loss: 33788.0938 - val_loss: 33705.9844
Epoch 5/10000
5/5 [==============================] - 0s 4ms/step - loss: 19036.9805 - val_loss: 17109.9434
Epoch 6/10000
5/5 [==============================] - 0s 4ms/step - loss: 16023.2461 - val_loss: 29281.2188
Epoch 7/10000
5/5 [==============================] - 0s 4ms/step - loss: 19703.3125 - val_loss: 19299.2500
Epoch 8/10000
5/5 [==============================] - 0s 4ms/step - loss: 17663.8477 - val_loss: 15617.1104
Epoch 9/10000
5/5 [==============================] - 0s 4ms/step - loss: 15620.6963 - val_loss: 17807.7910
Epoch 10/10000
5/5 [================

5/5 [==============================] - 0s 4ms/step - loss: 3499.4702 - val_loss: 3764.0959
Epoch 79/10000
5/5 [==============================] - 0s 4ms/step - loss: 3053.2522 - val_loss: 2717.9878
Epoch 80/10000
5/5 [==============================] - 0s 4ms/step - loss: 3871.6870 - val_loss: 3128.6211
Epoch 81/10000
5/5 [==============================] - 0s 4ms/step - loss: 2314.2947 - val_loss: 690.5169
Epoch 82/10000
5/5 [==============================] - 0s 4ms/step - loss: 1069.3535 - val_loss: 1000.7843
Epoch 83/10000
5/5 [==============================] - 0s 5ms/step - loss: 1152.8718 - val_loss: 829.4814
Epoch 84/10000
5/5 [==============================] - 0s 4ms/step - loss: 1090.1013 - val_loss: 1072.6023
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 1119.8466 - val_loss: 987.7377
Epoch 86/10000
5/5 [==============================] - 0s 4ms/step - loss: 1133.7267 - val_loss: 598.5381
Epoch 87/10000
5/5 [==============================] - 0s 4ms/step

5/5 [==============================] - 0s 4ms/step - loss: 520.0095 - val_loss: 428.7917
Epoch 157/10000
5/5 [==============================] - 0s 4ms/step - loss: 453.6446 - val_loss: 383.8856
Epoch 158/10000
5/5 [==============================] - 0s 4ms/step - loss: 519.1922 - val_loss: 750.9761
Epoch 159/10000
5/5 [==============================] - 0s 4ms/step - loss: 580.3488 - val_loss: 605.1085
Epoch 160/10000
5/5 [==============================] - 0s 4ms/step - loss: 691.3607 - val_loss: 372.3113
Epoch 161/10000
5/5 [==============================] - 0s 4ms/step - loss: 752.1157 - val_loss: 374.3541
Epoch 162/10000
5/5 [==============================] - 0s 4ms/step - loss: 837.3021 - val_loss: 852.0215
Epoch 163/10000
5/5 [==============================] - 0s 4ms/step - loss: 959.9126 - val_loss: 1550.7050
Epoch 164/10000
5/5 [==============================] - 0s 4ms/step - loss: 1400.6869 - val_loss: 463.7523
Epoch 165/10000
5/5 [==============================] - 0s 4ms/step - 

5/5 [==============================] - 0s 4ms/step - loss: 1415.4316 - val_loss: 770.2206
Epoch 234/10000
5/5 [==============================] - 0s 4ms/step - loss: 911.8644 - val_loss: 846.2910
Epoch 235/10000
5/5 [==============================] - 0s 4ms/step - loss: 1165.7079 - val_loss: 590.3621
Epoch 236/10000
5/5 [==============================] - 0s 4ms/step - loss: 1595.9313 - val_loss: 1319.4020
Epoch 237/10000
5/5 [==============================] - 0s 4ms/step - loss: 1780.0681 - val_loss: 1598.2277
Epoch 238/10000
5/5 [==============================] - 0s 4ms/step - loss: 1948.8357 - val_loss: 1901.1270
Epoch 239/10000
5/5 [==============================] - 0s 4ms/step - loss: 1385.1270 - val_loss: 1339.3210
Epoch 240/10000
5/5 [==============================] - 0s 4ms/step - loss: 1078.4402 - val_loss: 1351.7987
Epoch 241/10000
5/5 [==============================] - 0s 4ms/step - loss: 1181.6608 - val_loss: 1687.8859
Epoch 242/10000
5/5 [==============================] - 0s

Epoch 311/10000
5/5 [==============================] - 0s 4ms/step - loss: 667.2840 - val_loss: 332.1201
Epoch 312/10000
5/5 [==============================] - 0s 4ms/step - loss: 421.4134 - val_loss: 518.7028
Epoch 313/10000
5/5 [==============================] - 0s 4ms/step - loss: 612.9999 - val_loss: 512.5034
Epoch 314/10000
5/5 [==============================] - 0s 4ms/step - loss: 477.1662 - val_loss: 434.6696
Epoch 315/10000
5/5 [==============================] - 0s 4ms/step - loss: 552.2656 - val_loss: 218.2593
Epoch 316/10000
5/5 [==============================] - 0s 4ms/step - loss: 541.9565 - val_loss: 265.6550
Epoch 317/10000
5/5 [==============================] - 0s 4ms/step - loss: 448.2069 - val_loss: 203.2052
Epoch 318/10000
5/5 [==============================] - 0s 4ms/step - loss: 558.6938 - val_loss: 311.1777
Epoch 319/10000
5/5 [==============================] - 0s 4ms/step - loss: 552.2139 - val_loss: 475.4461
Epoch 320/10000
5/5 [==============================] - 

...

5/5 [==============================] - 0s 4ms/step - loss: 341.9852 - val_loss: 541.6426
Epoch 1322/10000
5/5 [==============================] - 0s 4ms/step - loss: 364.0670 - val_loss: 573.8846
Epoch 1323/10000
5/5 [==============================] - 0s 4ms/step - loss: 389.2299 - val_loss: 445.1077
Epoch 1324/10000
5/5 [==============================] - 0s 4ms/step - loss: 344.3211 - val_loss: 369.8857
Epoch 1325/10000
5/5 [==============================] - 0s 4ms/step - loss: 361.8509 - val_loss: 388.5826
Epoch 1326/10000
5/5 [==============================] - 0s 4ms/step - loss: 317.0695 - val_loss: 393.4939
Epoch 1327/10000
5/5 [==============================] - 0s 4ms/step - loss: 347.3255 - val_loss: 356.3837
Epoch 1328/10000
5/5 [==============================] - 0s 4ms/step - loss: 356.4507 - val_loss: 337.0711
Epoch 1329/10000
5/5 [==============================] - 0s 4ms/step - loss: 331.2165 - val_loss: 435.9428
Epoch 1330/10000
5/5 [==============================] - 0s 4ms/

5/5 [==============================] - 0s 4ms/step - loss: 314.2141 - val_loss: 208.3299
Epoch 1399/10000
5/5 [==============================] - 0s 4ms/step - loss: 334.0910 - val_loss: 199.2609
Epoch 1400/10000
5/5 [==============================] - 0s 4ms/step - loss: 344.4301 - val_loss: 282.6612
Epoch 1401/10000
5/5 [==============================] - 0s 4ms/step - loss: 358.8521 - val_loss: 234.3660
Epoch 1402/10000
5/5 [==============================] - 0s 4ms/step - loss: 366.1415 - val_loss: 151.0260
Epoch 1403/10000
5/5 [==============================] - 0s 4ms/step - loss: 317.2142 - val_loss: 169.5778
Epoch 1404/10000
5/5 [==============================] - 0s 4ms/step - loss: 320.5229 - val_loss: 174.5671
Epoch 1405/10000
5/5 [==============================] - 0s 4ms/step - loss: 291.7531 - val_loss: 179.1744
Epoch 1406/10000
5/5 [==============================] - 0s 4ms/step - loss: 338.4087 - val_loss: 169.5276
Epoch 1407/10000
5/5 [==============================] - 0s 4ms/

5/5 [==============================] - 0s 4ms/step - loss: 327.8579 - val_loss: 214.3385
Epoch 1476/10000
5/5 [==============================] - 0s 4ms/step - loss: 275.2063 - val_loss: 204.1894
Epoch 1477/10000
5/5 [==============================] - 0s 4ms/step - loss: 320.2593 - val_loss: 435.3123
Epoch 1478/10000
5/5 [==============================] - 0s 4ms/step - loss: 504.9590 - val_loss: 188.6311
Epoch 1479/10000
5/5 [==============================] - 0s 4ms/step - loss: 392.9966 - val_loss: 309.2244
Epoch 1480/10000
5/5 [==============================] - 0s 4ms/step - loss: 397.0123 - val_loss: 213.1206
Epoch 1481/10000
5/5 [==============================] - 0s 4ms/step - loss: 348.3903 - val_loss: 254.5066
Epoch 1482/10000
5/5 [==============================] - 0s 4ms/step - loss: 346.3015 - val_loss: 334.4500
Epoch 1483/10000
5/5 [==============================] - 0s 4ms/step - loss: 355.3903 - val_loss: 272.0212
Epoch 1484/10000
5/5 [==============================] - 0s 4ms/

5/5 [==============================] - 0s 4ms/step - loss: 304.3364 - val_loss: 337.2284
Epoch 1553/10000
5/5 [==============================] - 0s 4ms/step - loss: 283.7733 - val_loss: 245.2729
Epoch 1554/10000
5/5 [==============================] - 0s 4ms/step - loss: 257.7415 - val_loss: 379.0097
Epoch 1555/10000
5/5 [==============================] - 0s 4ms/step - loss: 267.1216 - val_loss: 281.9626
Epoch 1556/10000
5/5 [==============================] - 0s 4ms/step - loss: 284.2518 - val_loss: 290.5987
Epoch 1557/10000
5/5 [==============================] - 0s 4ms/step - loss: 342.8638 - val_loss: 343.5930
Epoch 1558/10000
5/5 [==============================] - 0s 4ms/step - loss: 296.2300 - val_loss: 411.3531
Epoch 1559/10000
5/5 [==============================] - 0s 4ms/step - loss: 286.8618 - val_loss: 384.2901
Epoch 1560/10000
5/5 [==============================] - 0s 4ms/step - loss: 308.8291 - val_loss: 331.7415
Epoch 1561/10000
5/5 [==============================] - 0s 4ms/

5/5 [==============================] - 0s 4ms/step - loss: 341.5768 - val_loss: 204.8458
Epoch 1630/10000
5/5 [==============================] - 0s 4ms/step - loss: 323.6663 - val_loss: 218.2839
Epoch 1631/10000
5/5 [==============================] - 0s 4ms/step - loss: 301.7525 - val_loss: 204.6677
Epoch 1632/10000
5/5 [==============================] - 0s 4ms/step - loss: 300.1492 - val_loss: 205.6833
Epoch 1633/10000
5/5 [==============================] - 0s 4ms/step - loss: 362.8982 - val_loss: 251.5802
Epoch 1634/10000
5/5 [==============================] - 0s 4ms/step - loss: 321.7129 - val_loss: 229.7201
Epoch 1635/10000
5/5 [==============================] - 0s 4ms/step - loss: 343.0766 - val_loss: 335.0283
Epoch 1636/10000
5/5 [==============================] - 0s 4ms/step - loss: 334.0487 - val_loss: 305.4274
Epoch 1637/10000
5/5 [==============================] - 0s 4ms/step - loss: 304.0446 - val_loss: 297.0242
Epoch 1638/10000
5/5 [==============================] - 0s 4ms/

In [22]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = mae_mape_calculator(trained_model, test_input, test_target, 168)

1/1 [==============================] - 0s 47ms/step
Month-1 - Error: [[50.986954]]
1/1 [==============================] - 0s 12ms/step
Month-2 - Error: [[21.685028]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[44.676865]]
1/1 [==============================] - 0s 12ms/step
Month-4 - Error: [[20.723831]]
1/1 [==============================] - 0s 12ms/step
Month-5 - Error: [[5.780075]]
1/1 [==============================] - 0s 12ms/step
Month-6 - Error: [[6.096634]]
1/1 [==============================] - 0s 12ms/step
Month-7 - Error: [[33.52365]]
1/1 [==============================] - 0s 13ms/step
Month-8 - Error: [[26.302444]]
1/1 [==============================] - 0s 12ms/step
Month-9 - Error: [[42.06888]]
1/1 [==============================] - 0s 12ms/step
Month-10 - Error: [[27.614594]]
1/1 [==============================] - 0s 12ms/step
Month-11 - Error: [[8.818176]]
1/1 [==============================] - 0s 12ms/step
Month-12 - Error: [[44.85518]]
1/1 [==

In [24]:
display(mae)
display(mape)

45.71407

0.16908966

In [25]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [26]:
errors, mae, mape = year_mae_mape_calculator(trained_model, test_input, test_target, 168)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2929.8616]] - Target[3060.1059999999998]| =  Error: [[130.24438]]; MAPE:[[0.04256205]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2556.6997]] - Target[3013.2439999999997]| =  Error: [[456.5442]]; MAPE:[[0.15151252]]
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[2471.8691]] - Target[3254.4680000000003]| =  Error: [[782.5989]]; MAPE:[[0.24046907]]


[array([[130.24438]], dtype=float32),
 array([[456.5442]], dtype=float32),
 array([[782.5989]], dtype=float32)]

456.4625

0.14484788